# IPBME6UE: Problem Set 2 - Medical Image Processing

## Spring 2023

## Group Members

**Person 1:**

YOUR NAME HERE

**Person 2:**

YOUR NAME HERE

Welcome to Problem Set 2!

In this problem set, we will be exploring some fundamental concepts in digital image processing and will perform segmentation and classification of tumors and leisons in imaging.

Breast cancer (BC) is the most commonly diagnosed malignant disease and [the leading cause of cancer death in women in Austria](https://pubmed.ncbi.nlm.nih.gov/35487918/).

Your task is to utilize machine learning to study mammograms in any way you want (e.g. classification, segmentation) as long as you justify why it is useful to do whatever it is you want to do. Choose something that interests you. As the adage goes, "do what you love, and you’ll never have to work another day in your life, or at least in our course..."

Treat this as a mini-project. We highly encourage working with 1 other person.


## Problem 1: Image Segmentation (30 points)


Download the [segmentation dataset (~200 MB)](https://imcfhkrems.sharepoint.com/:u:/r/teams/LV_62825/Class%20Materials/Exercise%202/Dataset_BUSI_with_GT.zip?csf=1&web=1&e=fpNqjR).

The dataset is divided into benign, malignant, and normal ultrasound imaging with binary masking.

**a.** Your first task is to segment the images. You may use any of the segmentation algorithms we saw in class. You can ignore the tumor-type division and use all the images but keep track of the method's performance per tumor type for the report in the next section. For the adventurist in you, try using deep learning to generate an initial ROI and active contours to shape that initial ROI into the final segmentation contour. Evaluate your results using an appropriate segmentation metric compared to the ground-truth labels as the target.


In [ ]:
### WRITE CODE IN HERE. You can have up to 3 cells for this question ################################












######################################################################################################

**b.** Create a report of your segmentation efforts.
Try answering the following questions:
- Which methods did you try? Can you reason your choices? 
- Which method worked the best for you? Which method worked the worst? Can you think why?
- How well did your methods work per class (benign, malignant, and normal)? 

**Answer:**

## Problem 2: Geometric Features (21 points)

As you may know, the geometric features of a tumor can sometimes reveal significant information about whether it is *benign* or *malignant*. [Radiologists frequently assess the geometry of lesions in mammograms when attempting to diagnose breast cancer](https://www.sciencedirect.com/science/article/pii/S2211568413003872).

In this exercise, we will attempt to explore some quantitative methods of differentiating between certain lesion shapes.

Download the two mammograms from Teams: `LEFT_CC_BENIGN.tif` depicts a cranial-caudal view (looking down on the breast from the patient's perspective) of the left breast. There is a benign lesion present in the mammogram. Can you locate it? 

If you are having difficulty locating the lesion, try seeing if you can spot it in the mask `LEFT_CC_BENIGN_MASK.tif`, which consists of a rough segmentation of the lesion. `RIGHT_CC_MALIGNANT.tif` and `RIGHT_CC_MALIGNANT_MASK.tif` are the mammogram and corresponding lesion segmentation of a different patient, but this mammogram depicts a breast with a **malignant** tumor.

**a.** Start by loading these two mammograms and their corresponding masks as image matrices. 
Plot them on a single figure (e.g., using a 2-by-2 grid of subplots). You should use reasonable **x** and **y** limits in your figures to get a reasonable depiction of your lesion segmentations.

Qualitatively describe the shape differences between the two lesions. What types of shapes and margins are indicative of malignant tumors?

**Answer:**

In [ ]:
### WRITE CODE IN HERE. You can have up to 2 cells for this question, but only one is required #######






######################################################################################################


**b.** Next, try computing some basic geometric features of the two lesions. Which features are the most significant for differentiating between them? Is this what you expected? The `regionprops()` function in the `skimage` library will automatically compute several geometric features for you, but you might choose to code up some of your own. If you don't trust the provided segmentations, feel free to create your segmentation method or modify the provided ones to get a better estimate of the lesion shapes! Finally, it could be interesting to apply some edge-detection filters to the mammograms to better emphasize the margins. 

You can also test your approach on other mammograms from a [larger dataset (~20 GB)](https://imcfhkrems.sharepoint.com/:f:/r/teams/LV_62825/Class%20Materials/Exercise%202?csf=1&web=1&e=X41LBH). 

**There is no single correct answer for this exercise, and you should definitely try several different approaches and explore!**

In [ ]:
### WRITE CODE IN HERE. You can have up to 3 cells for this question ################################












######################################################################################################

**c.** Computing GLCMs over the entire image is generally not very useful. Any subtle changes in lesion texture could make a significant difference in terms of the radiologist's diagnosis, but it might only result in a minimal change in the image's GLCM. A simple solution is to break the image down into chunks and compute a localized GLCM of each image chunk.

Try doing this on *one* of the provided mammograms. Iterate through the full mammogram and extract overlapping chunks (similar to what you would do when computing a 2D convolution), then compute the GLCM of each chunk. From each GLCM, compute your favorite Haralick texture feature and save it to a separate matrix. If you've done this correctly, you should be able to produce a textural feature map that is about the same size as your image. Plot this texture map alongside the original image. Does it look like what you expect? Why or why not?

**Tips:**
1. This could take really long ~ > 10 mins. If your code is taking too long to run, try downsampling the mammogram to a more reasonable size.
2. Don't worry too much about how the kernel behaves near the edges of the image, since these image regions are generally not as important.
3. You can choose whatever offsets you'd like when computing the GLCM. It is typical for researchers to make their textural features rotation-invariant by averaging features from multiple GLCMs computed using offsets oriented in all directions, and you can choose to do this if you want.
4. Explore different kernel sizes!

In [ ]:
### WRITE CODE IN HERE. You can have up to 3 cells for this question ################################












######################################################################################################

## Problem 3: Tumor Classification (49 Points)

In addition to the [mammograms themselves (~20 GB)](https://imcfhkrems.sharepoint.com/:f:/r/teams/LV_62825/Class%20Materials/Exercise%202?csf=1&web=1&e=X41LBH), which is based on [this](https://wiki.cancerimagingarchive.net/display/Public/CBIS-DDSM) the dataset includes "ground-truth" segmentations and `mass_case_description_train_set.csv`, which contains metadata information about mass shapes, mass margins, assessment numbers, pathology diagnoses, and subtlety in the data. Take some time to research what all of these different fields mean and how you might utilize them in your work. You dont need to use all of what is provided to you.

Some ideas:

1. Use the ROI’s or segmentations to extract features, and then train a classifier based on these feature table using the algorithms presented to you last year in the machine learning lectures (doesn't need to use deep learning).

2. Use convolutional neural networks. Feel free to use any package or use your own (custom code, sklearn, keras, Tensorflow, pytorch, timm etc.). If you dont want to place helper functions and classes into this notebook, place them in a .py file in the same folder called helperfunctions.py and import them into this notebook.

The delivery should be a report which includes the experiment planning, the extracted features and the results.
Try answering in your report the following questions:
- Which method(s) did you use? 
- Why did you choose these method(s)?
- What were your expectations towards these methods?
- How did they perform compared to your expectations?
- Which metrics did you use? To which ones did you pay the most attention?
- How do you feel about your results? 
- How would you plan to improve your results further?  

In [ ]:
### WRITE CODE IN HERE. ##############################################################################












######################################################################################################